HOLDOUT DATASET FOR HOLDOUT SCORE

In [7]:
bundle = joblib.load("group8_model_bundle.pkl")
bundle = joblib.load("group8_model_bundle.pkl")

preprocess_cluster = bundle["preprocess_cluster"]
pca = bundle["pca"]
kmeans = bundle["kmeans"]
xgb_pipeline = bundle["xgb_pipeline"]

type(preprocess_cluster)
type(pca)
type(kmeans)
type(xgb_pipeline)



sklearn.pipeline.Pipeline

DATA CLEANING SAME AS MODEL

In [16]:
import pandas as pd
import numpy as np

# Load holdout
holdout = pd.read_csv("retentiondata_case_holdout.csv")

def clean_features_only(df: pd.DataFrame):
    X = df.copy()

    # standardizing missing + trim strings
    X = X.replace({"NA": np.nan, "NaN": np.nan, "N/A": np.nan, "na": np.nan, "": np.nan})
    for c in X.select_dtypes(include=["object", "string"]).columns:
        X[c] = X[c].astype("string").str.strip()

    # dropping IDs
    X = X.drop(columns=[c for c in ["acct_ref", "cust_ref"] if c in X.columns], errors="ignore")

    # refunds_total -> binary received_refund
    if "refunds_total" in X.columns:
        X["refunds_total"] = pd.to_numeric(X["refunds_total"], errors="coerce").fillna(0)
        X["received_refund"] = (X["refunds_total"] > 0).astype(int)
        X = X.drop(columns=["refunds_total"], errors="ignore")
    else:
        # ensure column exists if training created it
        X["received_refund"] = 0

    # standardizing service tokens
    service_like = [
        "multi_line", "add_on_security", "add_on_backup", "add_on_protection",
        "tech_support_std", "stream_tv", "stream_movies", "stream_music", "premium_support"
    ]
    for c in [c for c in service_like if c in X.columns]:
        X[c] = X[c].replace({"No internet service": "No", "No phone service": "No"})

    # home_phone -> 0/1 (safe)
    if "home_phone" in X.columns:
        X["home_phone"] = (
            X["home_phone"]
            .replace({"No phone service": "No"})
            .map({"Yes": 1, "No": 0})
            .fillna(0)
            .astype(int)
        )

    # converting key numeric columns if present
    num_cols = [
        "tenure_mo", "monthly_fee", "total_billed", "age_years",
        "dependents_count", "referrals_count", "avg_long_dist_fee",
        "avg_gb_download", "extra_data_fees_total", "long_dist_fees_total"
    ]
    for c in [c for c in num_cols if c in X.columns]:
        X[c] = pd.to_numeric(X[c], errors="coerce")

    return X

# Apply cleaning
X_hold_base = clean_features_only(holdout)

# quick check
X_hold_base.shape


(1407, 34)

FEATURE ENGINEERING SAME AS MODEL

In [17]:
def engineer_features(X: pd.DataFrame):
    X = X.copy()

    # tenure featureS
    if "tenure_mo" in X.columns:
        X["tenure_mo"] = pd.to_numeric(X["tenure_mo"], errors="coerce")
        X["tenure_log1p"] = np.log1p(X["tenure_mo"].fillna(0))

        bins = [-np.inf, 6, 12, 24, 48, 72, np.inf]
        labels = ["0-6", "7-12", "13-24", "25-48", "49-72", "73+"]
        X["tenure_group"] = pd.cut(X["tenure_mo"], bins=bins, labels=labels)

    # bundle_score (count of Yes add-ons/streaming/support)
    service_cols = [
        "add_on_security", "add_on_backup", "add_on_protection",
        "tech_support_std", "premium_support",
        "stream_tv", "stream_movies", "stream_music"
    ]
    service_cols = [c for c in service_cols if c in X.columns]

    def to_binary_service(s):
        s = s.astype("string").replace({"No internet service": "No", "No phone service": "No"})
        return s.map({"Yes": 1, "No": 0}).fillna(0).astype(int)

    for c in service_cols:
        X[c] = to_binary_service(X[c])

    if service_cols:
        X["bundle_score"] = X[service_cols].sum(axis=1)
        # drop originals to reduce redundancy (recommended)
        X = X.drop(columns=service_cols, errors="ignore")
    else:
        X["bundle_score"] = 0

    # autopay flag from pay_method 
    if "pay_method" in X.columns:
        pm = X["pay_method"].astype("string").fillna("")
        X["is_autopay"] = pm.str.contains("automatic", case=False, na=False).astype(int)
    else:
        X["is_autopay"] = 0

    # penalties
    for c in ["extra_data_fees_total", "long_dist_fees_total"]:
        if c in X.columns:
            X[c] = pd.to_numeric(X[c], errors="coerce").fillna(0)
        else:
            X[c] = 0

    X["penalty_fees_total"] = X["extra_data_fees_total"] + X["long_dist_fees_total"]

    # penalty rate (safe divide) 
    if "total_billed" in X.columns:
        denom = pd.to_numeric(X["total_billed"], errors="coerce").fillna(0).replace(0, 1)
        X["penalty_fee_rate"] = X["penalty_fees_total"] / denom

    # billing per month features
    if "total_billed" in X.columns and "monthly_fee" in X.columns and "tenure_mo" in X.columns:
        t = X["tenure_mo"].fillna(0).replace(0, 1)
        X["avg_billed_per_month"] = pd.to_numeric(X["total_billed"], errors="coerce").fillna(0) / t
        X["fee_minus_avg_billed"] = pd.to_numeric(X["monthly_fee"], errors="coerce").fillna(0) - X["avg_billed_per_month"]

    # Missing flag 
    if "recent_offer" in X.columns:
        X["recent_offer_missing"] = X["recent_offer"].isna().astype(int)
    else:
        X["recent_offer_missing"] = 0

    # Handling Missing Values
    # Categorical
    if "recent_offer" in X.columns:
        X["recent_offer"] = X["recent_offer"].fillna("Unknown")

    if "internet_tech" in X.columns:
        X["internet_tech"] = X["internet_tech"].fillna("Unknown")

    # Numeric
    if "total_billed" in X.columns:
        X["total_billed"] = X["total_billed"].fillna(X["total_billed"].median())

    return X
    
X_hold_fe = engineer_features(X_hold_base)

# quick check
X_hold_fe.shape
X_hold_fe.head()


,has_dependents,tenure_mo,home_phone,multi_line,internet_plan,contract_term,e_bill_opt_in,pay_method,monthly_fee,total_billed,...,received_refund,tenure_log1p,tenure_group,bundle_score,is_autopay,penalty_fees_total,penalty_fee_rate,avg_billed_per_month,fee_minus_avg_billed,recent_offer_missing
0,No,8,1,Yes,Fiber optic,Month-to-month,Yes,Electronic check,99.65,820.50,...,0,2.197225,7-12,4,0,97.20,0.118464,102.562500,-2.912500,1
1,Yes,10,1,No,DSL,Month-to-month,No,Credit card (automatic),55.20,528.35,...,0,2.397895,7-12,3,1,94.10,0.178102,52.835000,2.365000,1
2,No,58,1,Yes,DSL,Two year,Yes,Credit card (automatic),59.90,3505.10,...,0,4.077537,49-72,3,1,2840.84,0.810488,60.432759,-0.532759,0
3,Yes,1,0,No,DSL,Month-to-month,No,Electronic check,30.20,30.20,...,0,0.693147,0-6,1,0,0.00,0.000000,30.200000,0.000000,1
4,Yes,17,1,No,DSL,Month-to-month,Yes,Mailed check,64.70,1093.10,...,0,2.890372,13-24,3,0,689.35,0.630638,64.300000,0.400000,1


In [19]:
# preprocess for clustering 
X_hold_cluster_processed = preprocess_cluster.transform(X_hold_fe)

# converting sparse
if hasattr(X_hold_cluster_processed, "toarray"):
    X_hold_cluster_processed = X_hold_cluster_processed.toarray()

# PCA transforming 
X_hold_pca = pca.transform(X_hold_cluster_processed)

# predicting clusters
holdout_clusters = kmeans.predict(X_hold_pca)

# adding cluster feature 
X_hold_fe["cluster"] = holdout_clusters.astype(str)


SCORING HOLDOUT WITH XGBOOST PIPELINE

In [20]:
holdout_prob = xgb_pipeline.predict_proba(X_hold_fe)[:, 1]

holdout_scored = holdout.copy()
holdout_scored["churn_probability"] = holdout_prob

holdout_scored.head()


,acct_ref,cust_ref,has_dependents,tenure_mo,home_phone,multi_line,internet_plan,add_on_security,add_on_backup,add_on_protection,...,avg_long_dist_fee,internet_tech,avg_gb_download,premium_support,stream_music,unlimited_data_opt,refunds_total,extra_data_fees_total,long_dist_fees_total,churn_probability
0,ACCT316552,CUST36921357,No,8,Yes,Yes,Fiber optic,No,No,Yes,...,12.15,Cable,26,No,Yes,Yes,0.0,0,97.20,0.989131
1,ACCT825485,CUST36620628,Yes,10,Yes,No,DSL,No,No,Yes,...,9.41,Cable,69,Yes,No,Yes,0.0,0,94.10,0.355907
2,ACCT864200,CUST36841391,No,58,Yes,Yes,DSL,No,Yes,No,...,48.98,Cable,76,Yes,No,Yes,0.0,0,2840.84,0.165702
3,ACCT212862,CUST64450449,Yes,1,No,No phone service,DSL,No,Yes,No,...,0.00,DSL,58,No,No,Yes,0.0,0,0.00,0.991139
4,ACCT810870,CUST29231696,Yes,17,Yes,No,DSL,No,No,No,...,40.55,Cable,41,No,Yes,Yes,0.0,0,689.35,0.629007


In [21]:
holdout_scored.to_csv("group8_holdout_scored.csv", index=False)
print("Saved: group8_holdout_scored.csv")


Saved: group8_holdout_scored.csv


In [22]:
holdout_scored["churn_probability"].describe()


count    1407.000000
mean        0.334274
std         0.343824
min         0.000237
25%         0.015284
50%         0.181886
75%         0.651530
max         0.997483
Name: churn_probability, dtype: float64